In [1]:
import zipfile
import os

def extract_zip(zip_file, extract_dir):
    with zipfile.ZipFile(zip_file, 'r') as zip_ref:
        zip_ref.extractall(extract_dir)

def data_train(fn):
    with open(fn, 'r') as f:
        train = f.readlines()
    train = [x.strip() for x in train]
    return train

train = data_train('vietnamese_tone_prediction\\train.txt')
print('Length of train:', len(train))
print('First 5 train:', train[:5])

Length of train: 2126280
First 5 train: ['Bộ phim lần đầu được công chiếu tại liên hoan phim Rome 2007 và sau đó được chiếu ở Fairbanks, Alaska ngày 21 tháng 9 năm 2007.', 'Những kiểu áo sơ mi may theo chất liệu cotton, KT, hay có chút co giãn năm nay cũng được các bạn trẻ ưa chuộng.', 'Đương kim tổng thống là Andrés Manuel López Obrador, người nhậm chức vào ngày 1 tháng 12 năm 2018.', 'Centaurea gloriosa là một loài thực vật có hoa trong họ Cúc.', 'Sau này mới thấy người ta nói, đó là con rắn đực đi tìm người ăn thịt để trả thù cho rắn cái, bà T cho biết thêm.']


In [2]:
import codecs
import csv
import re
import sys

def remove_tone_line(str):
    intab_l = "áàảãạâấầẩẫậăắằẳẵặóòỏõọôốồổỗộơớờởỡợéèẻẽẹêếềểễệúùủũụưứừửữựíìỉĩịýỳỷỹỵđ"
    intab_u = "ÁÀẢÃẠÂẤẦẨẪẬĂẮẰẲẴẶÓÒỎÕỌÔỐỒỔỖỘƠỚỜỞỠỢÉÈẺẼẸÊẾỀỂỄỆÚÙỦŨỤƯỨỪỬỮỰÍÌỈĨỊÝỲỶỸỴĐ"
    intab = intab_l + intab_u
    
    outtab_l = "a"*17 + "o"*17 + "e"*11 + "u"*11 + "i"*5 + "y"*5 + "d"
    outtab_u = "A"*17 + "O"*17 + "E"*11 + "U"*11 + "I"*5 + "Y"*5 + "D"
    outtab = outtab_l + outtab_u
    
    r = re.compile("|".join(intab))
    replaces_dict = dict(zip(intab, outtab))
    return r.sub(lambda m: replaces_dict[m.group(0)], str)

remove_tone_line('Đi một ngày đàng học 1 sàng khôn')

def normalizeString(s):
    marks = '[.!?,-${}()]'
    r = "(["+"\\".join(marks)+"])"
    s = re.sub(r, r" \1 ", s)
    s = re.sub(r"\s+", r" ", s).strip()
    return s

normalizeString('vui vẻ, hòa đồng, hoạt bát')

C:\Users\Thanh\AppData\Local\Temp\ipykernel_11676\2709821622.py:24: FutureWarning: Possible nested set at position 2
  s = re.sub(r, r" \1 ", s)


'vui vẻ , hòa đồng , hoạt bát'

In [3]:
train = [normalizeString(x) for x in train]
train_no_tone = [remove_tone_line(x) for x in train]
print('First 5 train:', train[:5])
print('First 5 train_no_tone:', train_no_tone[:5])

First 5 train: ['Bộ phim lần đầu được công chiếu tại liên hoan phim Rome 2007 và sau đó được chiếu ở Fairbanks , Alaska ngày 21 tháng 9 năm 2007 .', 'Những kiểu áo sơ mi may theo chất liệu cotton , KT , hay có chút co giãn năm nay cũng được các bạn trẻ ưa chuộng .', 'Đương kim tổng thống là Andrés Manuel López Obrador , người nhậm chức vào ngày 1 tháng 12 năm 2018 .', 'Centaurea gloriosa là một loài thực vật có hoa trong họ Cúc .', 'Sau này mới thấy người ta nói , đó là con rắn đực đi tìm người ăn thịt để trả thù cho rắn cái , bà T cho biết thêm .']
First 5 train_no_tone: ['Bo phim lan dau duoc cong chieu tai lien hoan phim Rome 2007 va sau do duoc chieu o Fairbanks , Alaska ngay 21 thang 9 nam 2007 .', 'Nhung kieu ao so mi may theo chat lieu cotton , KT , hay co chut co gian nam nay cung duoc cac ban tre ua chuong .', 'Duong kim tong thong la Andres Manuel Lopez Obrador , nguoi nham chuc vao ngay 1 thang 12 nam 2018 .', 'Centaurea gloriosa la mot loai thuc vat co hoa trong ho Cuc .', 

### Load and Trim data

In [4]:
PAD_token = 0 # Used for padding short sentences
SOS_token = 1 # Start-of-sentence token
EOS_token = 2 # End-of-sentence token

class Voc:
    def __init__(self, name):
        self.name = name
        self.trimmed = False
        self.word2index = {}
        self.word2count = {}
        self.index2word = {PAD_token: "PAD", SOS_token: "SOS", EOS_token: "EOS"}
        self.num_words = 3
        
    def addSentence(self, sentence):
        for word in sentence.split(' '):
            self.addWord(word)
    
    def addWord(self, word): 
        if word not in self.word2index:
            self.word2index[word] = self.num_words
            self.word2count[word] = 1
            self.index2word[self.num_words] = word
            self.num_words += 1
        else:
            self.word2count[word] += 1
            
    def trim(self, min_count):
        if self.trimmed:
            return
        self.trimmed = True
        
        keep_words = []
        
        for k, v in self.word2count.items():
            if v >= min_count:
                keep_words.append(k)
        
        print('keep_words {} / {} = {:.4f}'.format(len(keep_words), len(self.word2index), len(keep_words) / len(self.word2index)))
        
        self.word2index = {}
        self.word2count = {}
        self.index2word = {PAD_token: "PAD", SOS_token: "SOS", EOS_token: "EOS"}
        self.num_words = 3
        
        for word in keep_words:
            self.addWord(word)

### Normalize data

In [5]:
def ngram(text, length=4):
    words = text.split(' ')
    ngrams = []
    if len(words) <= length:
        words = words + ['PAD']*(length-len(words))
        return [' '.join(words)]
    else:
        for i in range(len(words)-length+1):
            ngrams.append(' '.join(words[i:i+length]))
        return ngrams

print(ngram('vui vẻ hòa đồng hoạt bát', 4))
print(ngram('vui vẻ hòa', 5))
    

['vui vẻ hòa đồng', 'vẻ hòa đồng hoạt', 'hòa đồng hoạt bát']
['vui vẻ hòa PAD PAD']


In [6]:
import itertools

train_ngram = list(itertools.chain.from_iterable([ngram(x) for x in train]))
train_no_tone_ngram = list(itertools.chain.from_iterable([ngram(x) for x in train_no_tone]))

In [7]:
corpus = list(zip(train_ngram, train_no_tone_ngram))
print('First 5 corpus:', corpus[:5])

First 5 corpus: [('Bộ phim lần đầu', 'Bo phim lan dau'), ('phim lần đầu được', 'phim lan dau duoc'), ('lần đầu được công', 'lan dau duoc cong'), ('đầu được công chiếu', 'dau duoc cong chieu'), ('được công chiếu tại', 'duoc cong chieu tai')]


In [8]:
import unicodedata

MAX_LENGTH = 4  # Maximum sentence length to consider

# Turn a Unicode string to plain ASCII, thanks to
# https://stackoverflow.com/a/518232/2809427
def unicodeToAscii(s):
    return ''.join(
        c for c in unicodedata.normalize('NFD', s)
        if unicodedata.category(c) != 'Mn'
    )

# Lowercase, trim, and remove non-letter characters
def normalizeString(s):
    # Tách dấu câu nếu kí tự liền nhau
    s = re.sub(r"([.!?,\-\&\(\)\[\]])", r" \1 ", s)
    # Thay thế nhiều spaces bằng 1 space.
    s = re.sub(r"\s+", r" ", s).strip()
    return s

# Read query/response pairs and return a voc object
def readVocs(lines, corpus_name = 'corpus'):
    # Split every line into pairs and normalize
    pairs = [[normalizeString(str(s)) for s in l] for l in lines]
    voc = Voc(corpus_name)
    return voc, pairs

voc, pairs = readVocs(corpus)  
  
# Returns True iff both sentences in a pair 'p' are under the MAX_LENGTH threshold
def filterPair(p):
    # Input sequences need to preserve the last word for EOS token
    return len(p[0].split(' ')) < MAX_LENGTH and len(p[1].split(' ')) < MAX_LENGTH

# Filter pairs using filterPair condition
def filterPairs(pairs):
    return [pair for pair in pairs if filterPair(pair)]

# # Using the functions defined above, return a populated voc object and pairs list
def loadPrepareData(voc, pairs):
    print("Read {!s} sentence pairs".format(len(pairs)))
    # pairs = filterPairs(pairs)
    print("Trimmed to {!s} sentence pairs".format(len(pairs)))
    print("Counting words...")
    for pair in pairs:
        voc.addSentence(pair[0])
        voc.addSentence(pair[1])
    print("Counted words:", voc.num_words)
    return voc, pairs
  
# Load/Assemble voc and pairs
save_dir = os.path.join("data", "save")
voc, pairs = loadPrepareData(voc, pairs)
# Print some pairs to validate
print("\npairs:")
for pair in pairs[:10]:
    print(pair)